Helper functions

In [10]:
def show_each_action_occurance(dataset):
    sums = np.zeros(lenA)
    for sample in dataset:
        a = A.index(sample[1][0])
        sums[a]+=1
    print(f"Action occurances: {sums}")

def save_with_config(dataset, name):
    if name == 'D1':
        CM = CM1
    elif name == 'D2':
        CM = CM2
    elif name == 'D3':
        CM = CM3
    else: raise Exception("not the right name of dataset")
    config = {
        'A': A,
        'G': G,
        'Otypes': Otypes,
        'lenA': lenA,
        'lenG': lenG,
        'lenOtypes': lenOtypes,
        'CM': CM,
    }
    np.save(f"{os.path.expanduser('~')}/Documents/{name}", np.asanyarray({'dataset': dataset, 'config': config}, dtype=object))

# Task 10.9. Generate three datasets
### Ds. 1: dataset2_1 generator

- Direct mapping + scene + target object
- 11 Actions + 9 Gestures
- 10000 draws
- CM is:
    - `dataset2_1` 2D `[11,9]`
    - `dataset2_2` 3D `[3,11,9]`
    - `dataset2_3` 4D `[2,3,11,9]`
- types, scene and other features are not used
- scene: n objects - initialized randomly
- target object generation type: **random**

In [11]:
from submodules import Robots, Tasks, Users
from submodules.Objects import Object
from submodules.Scenes import Scene
from submodules.Actions import Actions
from submodules.Gestures import Gestures
from submodules.Users import Users
from model_noK import *
import numpy as np
from numpy import array as a
import os

In [12]:
A = Actions.A = ['move_up', 'move_left', 'move_down', 'move_right', 
                 'put', 'put_on', 'pour', 'pick_up', 'place', 'open', 'close']

G = Gestures.G = ['swipe_up', 'swipe_left', 'swipe_down', 'swipe_right', 
                  'five', 'grab', 'thumbsup', 'rotate', 'point']
Otypes = Object.all_types
lenA = len(A)
lenG = len(G)
lenOtypes = len(Otypes)
object_types_order_list = ['cup', 'drawer', 'object', 'cup', 'drawer', 'object', 'cup']

In [13]:
CM1 = np.zeros((lenA,lenG)) # actions x gestures
                    # up,left,dwn,rght,fiv,grab,thum,rota,poin
CM1[:,:] = np.array([[.7,  .0, .0,  .0, .3, .0,   .0,  .0,  .0], # move_up
                     [.0,  .7, .0,  .0, .1, .0,   .1,  .1,  .0], # move_left
                     [.0,  .0, .7,  .0, .2, .0,   .0,  .1,  .0], # move_down
                     [.0,  .0, .0,  .7, .3, .0,   .0,  .0,  .0], # move_right
                     [.0,  .0, .4,  .0, .2, .0,   .0,  .0,  .4], # put
                     [.0,  .0,  1,  .0, .0, .0,   .0,  .0,  .0], # put_on
                     [.0,  .0, .0,  .0, .0, .0,   .1,  .9,  .0], # pour
                     [.2,  .0, .0,  .0, .0, .7,   .1,  .0,  .0], # pick_up
                     [.0,  .0, .0,  .0, .0, .0,   .3,  .0,  .7], # place
                     [.5,  .0, .0,  .0, .0, .5,   .0,  .0,  .0], # open
                     [.0,  .0, .5,  .0, .0, .5,   .0,  .0,  .0]])# close

In [14]:
CM2 = np.zeros((3, lenA,lenG)) # object types x actions x gestures
# objtyp 0: cup
                        # up,left,dwn,rght,fiv,grab,thum,rota,poin
CM2[0,:,:] = np.array([[.7,  .0, .0,  .0, .3, .0,   .0,  .0,  .0], # move_up
                       [.0,  .8, .0,  .0, .0, .0,   .1,  .1,  .0], # move_left
                       [.0,  .0, .7,  .0, .2, .0,   .0,  .1,  .0], # move_down
                       [.0,  .0, .0,  .7, .3, .0,   .0,  .0,  .0], # move_right
                       [.0,  .0, .0,  .0, .0, .0,   .0,  .0,  .0], # put
                       [.0,  .0, .0,  .0, .0, .0,   .0,  .0,  .0], # put_on
                       [.0,  .0, .0,  .0, .0, .0,   .1,  .9,  .0], # pour
                       [.2,  .0, .0,  .0, .0, .7,   .1,  .0,  .0], # pick_up
                       [.0,  .0, .0,  .0, .0, .0,    1,  .0,  .0], # place
                       [.0,  .0, .0,  .0, .0, .0,   .0,  .0,  .0], # open
                       [.0,  .0, .0,  .0, .0, .0,   .0,  .0,  .0]])# close
# user 0, objtyp 1: drawer
                      # up,left,dwn,rght,fiv,grab,thum,rota,poin
CM2[1,:,:] = np.array([[.7,  .0, .0,  .0, .3, .0,   .0,  .0,  .0], # move_up
                       [.0,  .7, .0,  .0, .1, .0,   .1,  .1,  .0], # move_left
                       [.0,  .0, .7,  .0, .2, .0,   .0,  .1,  .0], # move_down
                       [.0,  .0, .0,  .7, .3, .0,   .0,  .0,  .0], # move_right
                       [.0,  .0, .4,  .0, .2, .0,   .0,  .0,  .4], # put
                       [.0,  .0, .5,  .0, .0, .0,   .0,  .0,  .5], # put_on
                       [.0,  .0, .0,  .0, .0, .0,   .0,  .0,  .0], # pour
                       [.0,  .0, .0,  .0, .0, .0,   .0,  .0,  .0], # pick_up
                       [.0,  .0, .0,  .0, .0, .0,   .3,  .0,  .7], # place
                       [.4,  .0, .0,  .0, .0, .4,   .2,  .0,  .0], # open
                       [.0,  .0, .5,  .0, .0, .5,   .0,  .0,  .0]])# close
# objtyp 2: object = box
                      # up,left,dwn,rght,fiv,grab,thum,rota,poin
CM2[2,:,:] = np.array([[.7,  .0, .0,  .0, .3, .0,   .0,  .0,  .0], # move_up
                       [.0,  .7, .0,  .0, .1, .0,   .1,  .1,  .0], # move_left
                       [.0,  .0, .7,  .0, .2, .0,   .0,  .1,  .0], # move_down
                       [.0,  .0, .0,  .7, .3, .0,   .0,  .0,  .0], # move_right
                       [.0,  .0, .4,  .0, .2, .0,   .0,  .0,  .4], # put
                       [.0,  .0, .5,  .0, .0, .0,   .0,  .0,  .5], # put_on
                       [.0,  .0, .0,  .0, .0, .0,   .0,  .0,  .0], # pour
                       [.2,  .0, .0,  .0, .0, .7,   .1,  .0,  .0], # pick_up
                       [.0,  .0, .0,  .0, .0, .0,   .3,  .0,  .7], # place
                       [.0,  .0, .0,  .0, .0, .0,   .0,  .0,  .0], # open
                       [.0,  .0, .0,  .0, .0, .0,   .0,  .0,  .0]])# close

In [15]:
CM3 = np.zeros((2,3, lenA,lenG)) # users x object types x actions x gestures
CM3[0,0,:,:] = np.array([[ 1,  .0, .0,  .0,  0, .0,   .0,  .0,  .0], # move_up
                         [.0,   1, .0,  .0, .0, .0,    0,   0,  .0], # move_left
                         [.0,  .0,  1,  .0,  0, .0,   .0,   0,  .0], # move_down
                         [.0,  .0, .0,   1,  0, .0,   .0,  .0,  .0], # move_right
                         [.0,  .0, .0,  .0, .0, .0,   .0,  .0,  .0], # put
                         [.0,  .0, .0,  .0, .0, .0,   .0,  .0,  .0], # put_on
                         [.0,  .0, .0,  .0, .0, .0,    0,   1,  .0], # pour
                         [.0,  .0, .0,  .0, .0,  1,    0,  .0,  .0], # pick_up
                         [.0,  .0, .0,  .0, .0, .0,    0,  .0,   1], # place
                         [.0,  .0, .0,  .0, .0, .0,   .0,  .0,  .0], # open
                         [.0,  .0, .0,  .0, .0, .0,   .0,  .0,  .0]])# close
# objtyp 1: drawer
                      # up,left,dwn,rght,fiv,grab,thum,rota,poin
CM3[0,1,:,:] = np.array([[ 1,  .0, .0,  .0,  0, .0,   .0,  .0,  .0], # move_up
                         [.0,   1, .0,  .0,  0, .0,    0,   0,  .0], # move_left
                         [.0,  .0,  1,  .0,  0, .0,   .0,   0,  .0], # move_down
                         [.0,  .0, .0,   1,  0, .0,   .0,  .0,  .0], # move_right
                         [.0,  .0,  0,  .0,  0, .0,   .0,  .0,   1], # put
                         [.0,  .0,  0,  .0, .0, .0,    1,  .0,   0], # put_on
                         [.0,  .0, .0,  .0, .0, .0,   .0,  .0,  .0], # pour
                         [.0,  .0, .0,  .0, .0, .0,   .0,  .0,  .0], # pick_up
                         [.0,  .0, .0,  .0, .0,  1,    0,  .0,   0], # place
                         [ 0,  .0, .0,  .0, .0,  0,    0,   1,  .0], # open
                         [.0,  .0,  0,  .0,  1,  0,   .0,  .0,  .0]])# close
# user 0, objtyp 2: object = box
                        # up,left,dwn,rght,fiv,grab,thum,rota,poin
CM3[0,2,:,:] = np.array([[ 1,  .0, .0,  .0,  0, .0,   .0,  .0,  .0], # move_up
                         [.0,   1, .0,  .0,  0, .0,    0,   0,  .0], # move_left
                         [.0,  .0,  1,  .0,  0, .0,   .0,   0,  .0], # move_down
                         [.0,  .0, .0,   1,  0, .0,   .0,  .0,  .0], # move_right
                         [.0,  .0,  0,  .0,  1, .0,   .0,  .0,   0], # put
                         [.0,  .0,  0,  .0, .0, .0,   .0,  .0,   1], # put_on
                         [.0,  .0, .0,  .0, .0, .0,   .0,  .0,  .0], # pour
                         [ 0,  .0, .0,  .0, .0,  1,    0,  .0,  .0], # pick_up
                         [.0,  .0, .0,  .0, .0, .0,    1,  .0,   0], # place
                         [.0,  .0, .0,  .0, .0, .0,   .0,  .0,  .0], # open
                         [.0,  .0, .0,  .0, .0, .0,   .0,  .0,  .0]])# close
# user 1, objtyp 0: cup
                        # up,left,dwn,rght,fiv,grab,thum,rota,poin
CM3[0,0,:,:] = np.array([[.7,  .0, .0,  .0, .3, .0,   .0,  .0,  .0], # move_up
                         [.0,  .8, .0,  .0, .0, .0,   .1,  .1,  .0], # move_left
                         [.0,  .0, .7,  .0, .2, .0,   .0,  .1,  .0], # move_down
                         [.0,  .0, .0,  .7, .3, .0,   .0,  .0,  .0], # move_right
                         [.0,  .0, .0,  .0, .0, .0,   .0,  .0,  .0], # put
                         [.0,  .0, .0,  .0, .0, .0,   .0,  .0,  .0], # put_on
                         [.0,  .0, .0,  .0, .0, .0,   .1,  .9,  .0], # pour
                         [.2,  .0, .0,  .0, .0, .7,   .1,  .0,  .0], # pick_up
                         [.0,  .0, .0,  .0, .0, .0,    1,  .0,  .0], # place
                         [.0,  .0, .0,  .0, .0, .0,   .0,  .0,  .0], # open
                         [.0,  .0, .0,  .0, .0, .0,   .0,  .0,  .0]])# close
# user 1, objtyp 1: drawer
                      # up,left,dwn,rght,fiv,grab,thum,rota,poin
CM3[0,1,:,:] = np.array([[.7,  .0, .0,  .0, .3, .0,   .0,  .0,  .0], # move_up
                         [.0,  .7, .0,  .0, .1, .0,   .1,  .1,  .0], # move_left
                         [.0,  .0, .7,  .0, .2, .0,   .0,  .1,  .0], # move_down
                         [.0,  .0, .0,  .7, .3, .0,   .0,  .0,  .0], # move_right
                         [.0,  .0, .4,  .0, .2, .0,   .0,  .0,  .4], # put
                         [.0,  .0, .5,  .0, .0, .0,   .0,  .0,  .5], # put_on
                         [.0,  .0, .0,  .0, .0, .0,   .0,  .0,  .0], # pour
                         [.0,  .0, .0,  .0, .0, .0,   .0,  .0,  .0], # pick_up
                         [.0,  .0, .0,  .0, .0, .0,   .3,  .0,  .7], # place
                         [.4,  .0, .0,  .0, .0, .4,   .2,  .0,  .0], # open
                         [.0,  .0, .5,  .0, .0, .5,   .0,  .0,  .0]])# close
# user 1, objtyp 2: object = box
                      # up,left,dwn,rght,fiv,grab,thum,rota,poin
CM3[0,2,:,:] = np.array([[.7,  .0, .0,  .0, .3, .0,   .0,  .0,  .0], # move_up
                         [.0,  .7, .0,  .0, .1, .0,   .1,  .1,  .0], # move_left
                         [.0,  .0, .7,  .0, .2, .0,   .0,  .1,  .0], # move_down
                         [.0,  .0, .0,  .7, .3, .0,   .0,  .0,  .0], # move_right
                         [.0,  .0, .4,  .0, .2, .0,   .0,  .0,  .4], # put
                         [.0,  .0, .5,  .0, .0, .0,   .0,  .0,  .5], # put_on
                         [.0,  .0, .0,  .0, .0, .0,   .0,  .0,  .0], # pour
                         [.2,  .0, .0,  .0, .0, .7,   .1,  .0,  .0], # pick_up
                         [.0,  .0, .0,  .0, .0, .0,   .3,  .0,  .7], # place
                         [.0,  .0, .0,  .0, .0, .0,   .0,  .0,  .0], # open
                         [.0,  .0, .0,  .0, .0, .0,   .0,  .0,  .0]])# close

In [16]:
draws = 10000
common_sense_probability_threshold = 0.9
feasibility_probability_threshold = 0.01

Dataset generation:
- D1
- D2
- D3

In [17]:
dataset_name = 'D1'

In [18]:
def create_observation(s,TaTo, dataset_name):
    target_object_id = s.O.index(TaTo[1])
    target_action_id = Actions.A.index(TaTo[0])
    target_object_type_id = Object.all_types.index(getattr(s, TaTo[1]).type)
    user_id = np.random.randint(2)
    
    if dataset_name == 'D1':
        gesture_probs_intent = CM1[target_action_id,:]
    elif dataset_name == 'D2':
        gesture_probs_intent = CM2[target_object_type_id,target_action_id,:]
    elif dataset_name == 'D3':
        gesture_probs_intent = CM3[user_id,target_object_type_id,target_action_id,:]
    if sum(gesture_probs_intent) == 0: return False, False
    
    performed_gesture = np.random.choice(np.arange(lenG),p = gesture_probs_intent,size=1)

    gesture_vec = np.zeros(lenG)
    gesture_vec[performed_gesture] = 1
    gesture_vec = abs(gesture_vec + np.random.normal(loc = 0, scale = 0.2,size =(1,lenG)))[0]
    gesture_vec = gesture_vec/np.sum(gesture_vec)
    
    focus_point = abs(np.random.normal(loc = s.objects[target_object_id].position, scale = 0.1,size =(1,3)))
    
    return gesture_vec, focus_point

In [20]:
dataset = []
# Generate random action
while len(dataset) < draws:

    n_objects = round(np.random.normal(3.5,1))
    init = ",".join(object_types_order_list[0:n_objects])
    
    s = Scene(init=init, random=True)
    # Random draw from possible actions
    TaTo = Actions.get_random_possible_action(s, ignore_location=True, p=common_sense_probability_threshold)
    if TaTo is False: continue
    
    gobs, oobs = create_observation(s, TaTo, dataset_name)
    if gobs is False: continue
    
    dataset.append((s.to_dict(), TaTo, gobs, oobs))
    #if i % 1000 == 0: print(i)

In [21]:
save_with_config(dataset, dataset_name)

In [22]:
show_each_action_occurance(dataset)

Action occurances: [   0. 1858. 2590. 1853.  481.  285.   39. 1029. 1024.  236.  605.]


Some actions were not feasible and were not triggered as often, this can be a problem, when evaluating dataset.
- D1 action occurances: [   0. 1838. 2564. 1848.  473.  276.   32. 1061. 1040.  214.  644.]
- D2 action occurances: [   0. 1854. 2504. 1913.  507.  253.   23. 1047. 1009.  221.  653.]
- D3 action occurances: [   0. 1850. 2566. 1854.  489.  257.   31. 1029. 1010.  248.  666.]